In [ ]:
import requests
import traceback

# Definindo a URL da API
url = "https://storage.googleapis.com/covid19-open-data/v3/latest/aggregated.csv"
dbfs_temp_file_path = "dbfs:/tmp/aggregated.csv"

try:
    # 1. Fazendo a requisição para obter os dados
    response = requests.get(url)
    response.raise_for_status()  # Verifica se houve algum erro na requisição

    # 2. Salvando o conteúdo em um arquivo temporário CSV no DBFS
    with open("/tmp/aggregated.csv", "wb") as file:
        file.write(response.content)
    
    # Movendo o arquivo local para o DBFS
    dbutils.fs.mv("file:/tmp/aggregated.csv", dbfs_temp_file_path)

    # 3. Leitura do arquivo CSV em um DataFrame Spark
    df_bronze = spark.read.csv(dbfs_temp_file_path, header=True, inferSchema=True)
    
    if df_bronze.rdd.isEmpty():
        raise ValueError("O arquivo CSV está vazio.")
    
    print("Arquivo CSV lido com sucesso no DataFrame Spark!")

    # 4. Carregamento dos Dados na Camada Bronze em formato Delta
    bronze_directory = "/covid/bronze" 

    # Criando o diretório, caso não exista
    dbutils.fs.mkdirs(bronze_directory)

    # Salvando os dados em formato Delta na camada Bronze
    df_bronze.write.format("delta").mode("overwrite").save(bronze_directory)
    print("Dados carregados na camada Bronze no formato Delta com sucesso!")

    # 5. Leitura dos dados carregados 
    df_bronze_loaded = spark.read.format("delta").load(bronze_directory)

    # Quantidade de linhas e colunas
    row_count = df_bronze_loaded.count()
    column_count = len(df_bronze_loaded.columns)

    print(f"Quantidade de linhas: {row_count}")
    print(f"Quantidade de colunas: {column_count}")

except requests.exceptions.RequestException as e:
    print(f"Erro ao baixar o arquivo CSV: {e}")
    print(traceback.format_exc())
    raise
except Exception as e:
    print(f"Erro ao processar o arquivo CSV: {e}")
    print(traceback.format_exc())
    raise
finally:
    # Removendo o arquivo temporário no DBFS
    dbutils.fs.rm(dbfs_temp_file_path)
    print("Arquivo temporário removido do DBFS.")


Arquivo CSV lido com sucesso no DataFrame Spark!
Dados carregados na camada Bronze no formato Delta com sucesso!
Formato do arquivo: delta
Quantidade de linhas: 22712
Quantidade de colunas: 737
Arquivo temporário removido do DBFS.
